In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Imagen 3 Image Generation

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/imagen3_image_generation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fvision%2Fgetting-started%2Fimagen3_image_generation.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/vision/getting-started/imagen3_image_generation.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/imagen3_image_generation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>


| | |
|-|-|
|Author(s) | [Katie Nguyen](https://github.com/katiemn) |

## Overview

### Imagen 3

Imagen 3 on Vertex AI brings Google's state of the art generative AI capabilities to application developers. Imagen 3 is Google's highest quality text-to-image model to date. It's capable of creating images with astonishing detail. Thus, developers have more control when building next-generation AI products that transform their imagination into high quality visual assets. Learn more about [Imagen on Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/image/overview).


In this tutorial, you will learn how to use the Vertex AI SDK for Python to interact with the Imagen 3 and Imagen 3 Fast models to generate images showcasing:

- Photorealistic scenes
- Text rendered within images
- Quality and latency comparisons within the two models

## Get started


### Install Vertex AI SDK for Python


In [ ]:
%pip install --upgrade --user google-cloud-aiplatform

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [17]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment.


In [10]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [11]:
PROJECT_ID = "*******"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

### Import libraries

In [3]:
from vertexai.preview.vision_models import ImageGenerationModel

### Define a helper function

In [4]:
import typing

import IPython.display
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps


def display_image(
    image,
    max_width: int = 600,
    max_height: int = 350,
) -> None:
    pil_image = typing.cast(PIL_Image.Image, image._pil_image)
    if pil_image.mode != "RGB":
        # RGB is supported by all Jupyter environments (e.g. RGBA is not yet)
        pil_image = pil_image.convert("RGB")
    image_width, image_height = pil_image.size
    if max_width < image_width or max_height < image_height:
        # Resize to display a smaller notebook image
        pil_image = PIL_ImageOps.contain(pil_image, (max_width, max_height))
    IPython.display.display(pil_image)

### Load the image generation models

Imagen 3: `imagen-3.0-generate-001`

Imagen 3 Fast: `imagen-3.0-fast-generate-001`

In [29]:
generation_model = ImageGenerationModel.from_pretrained("imagen-3.0-generate-001")
generation_model_fast = ImageGenerationModel.from_pretrained(
    "imagen-3.0-fast-generate-001"
)

In [34]:
import os
import time
import matplotlib.pyplot as plt

def read_descriptions(file_path, separator="-" * 50):
    """
    Reads descriptions from a file and splits them using the specified separator.
    """
    with open(file_path, "r") as file:
        content = file.read()
    return content.split(separator)

def update_file(file_path, remaining_descriptions, separator="-" * 50):
    """
    Updates the file with the remaining descriptions.
    """
    with open(file_path, "w") as file:
        file.write(separator.join(remaining_descriptions))

def generate_and_save_images(descriptions, file_path, output_folder, base_name="genai", separator="-" * 50):
    """
    Generates images based on descriptions, saves them in the specified folder, and updates the text file.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for i, prompt in enumerate(descriptions):
        if not prompt.strip():  # Skip empty prompts
            continue

        print(f"Generating image for description {i + 1}...")
        
        try:
            # Generate images with the model
            image = generation_model.generate_images(
                prompt="The painting depicts a rural scene at dusk or dawn.  A dirt road, muddy from recent rain, curves gently through the picture.  Bare, leafless trees line the road, their branches stark against a muted yellow-orange sky.  A woman, dressed in dark clothing and wearing a bonnet, walks along the road, away from the viewer.  In the background, a large stone wall runs alongside the road, culminating in a gated entrance to a large, stone building, possibly a manor house or estate. The overall atmosphere is tranquil and somewhat melancholic, with a focus on the muted colors and the solitary figure on the road.  The light suggests the time of day is either just before sunrise or just after sunset.",
                number_of_images=1,
                aspect_ratio="3:4",
                safety_filter_level="block_some",
                person_generation="all_ages",
            )[0]._pil_image

            # Save images with appropriate names
            image_name = f"{base_name}_model_m_{i + 1}.png"
            image.save(os.path.join(output_folder, image_name))
            print(f"Image saved as {image_name}")

            # Remove the processed description from the file only after successful generation
            remaining_descriptions = descriptions[i + 1 :]  # Get all unprocessed descriptions
            update_file(file_path, remaining_descriptions, separator)
            print(f"Updated file after processing description {i + 1}.")

            # Pause between generations to avoid overloading resources
            time.sleep(30)

        except Exception as e:
            print(f"Error generating image for description {i + 1}: {e}")
            # Skip updating the file and move to the next description
            continue

def main():
    # Path to the .txt file containing descriptions
    txt_file_path = "response_log.txt"  # Update this to your actual file path

    # Folder to save generated images
    output_folder = "fakeai"

    # Read descriptions
    descriptions = read_descriptions(txt_file_path)

    # Generate and save images
    generate_and_save_images(descriptions, txt_file_path, output_folder)

if __name__ == "__main__":
    main()


Generating image for description 1...
Image saved as genai_model_m_1.png
Updated file after processing description 1.


KeyboardInterrupt: 